# SyftBox Data Owner (DO) - Bob's Notebook

This notebook demonstrates the **Beaver + SyftBox integration** from the data owner's perspective.

Key features:
- Data is **encrypted at rest** using SyftBox crypto
- **Session-based** communication with data scientists
- Private keys are managed in `.sbc/` vault
- Files sync automatically via SyftBox server

## Setup

In [1]:
import sys
from pathlib import Path

# Add beaver to path (for development)
sys.path.insert(0, "../python/src")

import beaver
from beaver import Twin
import pandas as pd
import numpy as np

In [2]:
# Check SyftBox SDK availability
try:
    import syftbox_sdk
    print(f"✓ SyftBox SDK version: {syftbox_sdk.__version__}")
except ImportError:
    print("✗ SyftBox SDK not installed. Run: pip install syftbox-sdk")

✓ SyftBox SDK version: 0.1.5


## Connect with SyftBox Backend

Connect as a data owner. SyftBox mode is now the default.

In [3]:
# Current directory is the client's data directory
DATA_DIR = Path.cwd()
USER_EMAIL = "client1@sandbox.local"

print(f"Data directory: {DATA_DIR}")
print(f"User: {USER_EMAIL}")

Data directory: /Users/madhavajay/dev/biovault-beaver/workspace2/sandbox/client1@sandbox.local
User: client1@sandbox.local


In [4]:
# Connect with SyftBox backend (encrypted mode is now default)
bv = beaver.connect(
    user=USER_EMAIL,
    data_dir=DATA_DIR,
)

print(f"✓ Connected as {bv.user}")
print(f"  SyftBox enabled: {bv.syftbox_enabled}")
print(f"  Inbox path: {bv.inbox_path}")

🔄 Auto-load replies enabled for client1@sandbox.local (polling every 2.0s)
✓ Connected as client1@sandbox.local
  SyftBox enabled: True
  Inbox path: /Users/madhavajay/dev/biovault-beaver/workspace2/sandbox/client1@sandbox.local/datasites/client1@sandbox.local/shared/biovault


In [5]:
# Check encryption status
if bv.backend:
    print(f"  Encryption: {'✓ Enabled' if bv.backend.uses_crypto else '✗ Disabled'}")
    print(f"  Vault path: {bv.backend.vault_path}")
    print(f"  Shared path: {bv.backend.shared_path}")

  Encryption: ✓ Enabled
  Vault path: /Users/madhavajay/dev/biovault-beaver/workspace2/sandbox/client1@sandbox.local/.sbc
  Shared path: /Users/madhavajay/dev/biovault-beaver/workspace2/sandbox/client1@sandbox.local/datasites/client1@sandbox.local/shared/biovault


## Check for Session Requests

Data scientists must request a session before accessing data.
View pending requests and accept/reject them.

In [6]:
# List pending session requests
requests = bv.session_requests()
requests

SessionRequests: empty

In [7]:
# Accept a session request (if any)
if len(requests) > 0:
    # Accept the first request
    session = requests[0].accept()
    print(f"Session folder: {session.local_folder}")
else:
    print("No pending session requests.")
    print("Wait for a data scientist to request a session...")

No pending session requests.
Wait for a data scientist to request a session...


## Create Patient Data Twin

In [8]:
# Real patient data (SENSITIVE)
real_data = pd.DataFrame({
    'patient_id': ['P001', 'P002', 'P003', 'P004', 'P005'],
    'name': ['Alice Johnson', 'Bob Smith', 'Carol White', 'David Brown', 'Eve Davis'],
    'age': [34, 45, 29, 52, 38],
    'test_result': [7.2, 8.5, 6.1, 9.3, 7.8],
    'diagnosis': ['positive', 'positive', 'negative', 'positive', 'positive']
})

# Mock data (anonymized)
mock_data = pd.DataFrame({
    'patient_id': ['M001', 'M002', 'M003'],
    'name': ['Patient A', 'Patient B', 'Patient C'],
    'age': [30, 40, 35],
    'test_result': [6.5, 8.0, 7.0],
    'diagnosis': ['negative', 'positive', 'positive']
})

print("🔒 PRIVATE DATA:")
display(real_data)
print("\n🌍 PUBLIC DATA:")
display(mock_data)

🔒 PRIVATE DATA:


,patient_id,name,age,test_result,diagnosis
0,P001,Alice Johnson,34,7.2,positive
1,P002,Bob Smith,45,8.5,positive
2,P003,Carol White,29,6.1,negative
3,P004,David Brown,52,9.3,positive
4,P005,Eve Davis,38,7.8,positive



🌍 PUBLIC DATA:


,patient_id,name,age,test_result,diagnosis
0,M001,Patient A,30,6.5,negative
1,M002,Patient B,40,8.0,positive
2,M003,Patient C,35,7.0,positive


In [9]:
# Create Twin
patient_data = Twin(
    private=real_data,
    public=mock_data,
    owner=USER_EMAIL,
    name="patient_data"
)

patient_data

⚠️  Twin: patient_data (REAL + MOCK DATA)
  ⚠️  Be careful: This Twin contains both real and mock data
  🔒 Private      patient_id           name  age  test_result diagnosis
0...    ← .value uses this
  🌍 Public      patient_id       name  age  test_result diagnosis
0    ...    ✓
  Live: ⚫ Disabled
  IDs: twin=b03db1b3... private=24f654d1... public=ef37a17a...

## Publish Twin to Session

Once a session is accepted, publish data to the session folder.
Only the session peer can access this data.

In [10]:
patient_data.public

,patient_id,name,age,test_result,diagnosis
0,M001,Patient A,30,6.5,negative
1,M002,Patient B,40,8.0,positive
2,M003,Patient C,35,7.0,positive


In [11]:
patient_data.private

,patient_id,name,age,test_result,diagnosis
0,P001,Alice Johnson,34,7.2,positive
1,P002,Bob Smith,45,8.5,positive
2,P003,Carol White,29,6.1,negative
3,P004,David Brown,52,9.3,positive
4,P005,Eve Davis,38,7.8,positive


In [12]:
# Publish to session (if we have an active session)
try:
    session.remote_vars["patient_data"] = patient_data
    print(f"✓ Published patient_data to session {session.session_id}")
except NameError:
    print("No active session. Accept a session request first.")
    print("Alternatively, publish to global remote_vars:")
    print("  bv.remote_vars['patient_data'] = patient_data")

No active session. Accept a session request first.
Alternatively, publish to global remote_vars:
  bv.remote_vars['patient_data'] = patient_data


## Simple Counter Example

In [13]:
# Create a simple counter Twin for testing
counter = Twin(
    private=42,      # Real data (stays local, encrypted)
    public=0,        # Mock data (shareable)
    owner=USER_EMAIL,
    name="counter"
)

counter

⚠️  Twin: counter (REAL + MOCK DATA)
  ⚠️  Be careful: This Twin contains both real and mock data
  🔒 Private    42    ← .value uses this
  🌍 Public    0    ✓
  Live: ⚫ Disabled
  IDs: twin=abf84742... private=ad926357... public=f67526d7...

In [14]:
# Publish to session if available, otherwise to global registry
try:
    session.remote_vars["counter"] = counter
    print(f"✓ Published counter to session {session.session_id}")
except NameError:
    bv.remote_vars["counter"] = counter
    print("✓ Published counter to global registry")

🌍 Using PUBLIC data from Twin 'counter...'
📢 Published Twin 'counter' (public side available at: /Users/madhavajay/dev/biovault-beaver/workspace2/sandbox/client1@sandbox.local/datasites/client1@sandbox.local/shared/public/data/299b13169e9641c695e1eed0fb3ad260.beaver)
✓ Published counter to global registry


## Check Workspace

In [15]:
# View all shared data
bv.workspace()

Name,Owner,Type,Status,Last Action
counter,client1@sandbox.local,Twin[int],📤 Sent,0s ago


## Check Inbox for Computation Requests

In [16]:
# Check for computation requests from peers
bv.inbox()

InboxView(/Users/madhavajay/dev/biovault-beaver/workspace2/sandbox/client1@sandbox.local/datasites/client1@sandbox.local/shared/biovault): empty

## Summary

With session-based SyftBox integration:
- ✓ Data is encrypted at rest
- ✓ Session-based access control
- ✓ Only authorized peers can decrypt
- ✓ Private data never leaves this environment
- ✓ Files sync automatically via SyftBox server